crawler类（爬虫类）

In [28]:
import urllib2
from pysqlite2 import dbapi2 as sqlite
from bs4 import BeautifulSoup,Tag,CData
from urlparse import urljoin

class crawler:
    #初始化crawler类并传入数据库名称：
    def __init__(self,dbname):
        self.con = sqlite.connect(dbname)
    
    def __del__(self):
        self.con.close()
    
    def dbcommit(self):
        self.con.commit()
        
    
    #辅助函数，用于获取条目的ID，并且如果条目不存在，就将其加入数据库：
    def getentryid(self,table,field,value,createnew = True):
        cur = self.con.execute("select rowid from %s where %s='%S'" % (table,field,value))
        res = cur.fetchone()
        if res == None:
            cur = self.con.execute("insert into %s (%s) values ('%s') " % (table,field,value))
            return cur.lastrowid
        else:
            return res[0]
    
    #为每个网页建立索引
    def addtoindex(self,url,soup):
        if self.isindexed(url):return
        print 'Indexing %s'% url
        
        #获取每个单词
        text = self.gettextonly(soup)
        words = self.separatewords(text)
        
        #得到URL的id
        urlid = self.getentryid('urllist','url',url)
        
        #将每个单词与该url关联
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords:continue
            self.con.execute("insert into wordlocation(urlid,wordid,location) \ values (%d %d %d)" % (url,wordid,i))
        
    #从一个HTML网页中提取文字
    def gettextonly(self,soup):
        #string 获取标签里面的内容
        v = soup.string
        if v == None:
            # 可以将tag的节点以列表的方式输出
            c = soup.contents
            resulttext = ''
            for t in c :
                subtext = self.gettextonly(t)
                resulttext += subtext + '\n'
            return resulttext
        else:
            return v.strip()
        
    
    #如果URL已经建过索引，则返回true
    def isindexed(self,url):
        u = self.con.execute("select rowid from urllist where url = '%s'" % url).fetchone()
        if u != None:
            #检查它是否已经被检索过了
            v = self.con.execute('selet * from wordlocation where urlid = %d' % u[0]).fetchone()
            if v != None:return True
        return False
       
    #添加一个关联两个网页的链接
    def addlinkref(self,urlFrom,urlTo,linkText):
        pass
    
    #根据任何非空白字符进行分词
    def separatewords(self,text):
        splitter = re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s != '']
    
    #从一小组网页开始进行广度优先搜索，直至某一给定深度
    #期间为网页建立索引
    def crawl(self,pages,depth = 3):
        for i in range(depth):
            newpages = set()
            for page in pages:
                try :
                    c = urllib2.urlopen(page)
                except:
                    print "Could not open %s" % page
                    continue
                soup = BeautifulSoup(c.read(),"html5lib")
                self.addtoindex(page,soup)
                
                links = soup('a')
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url = urljoin(page,link['href'])
                        if url.find("'")!=-1:continue
                        url = url.split('#')[0] #去掉位置部分
                        if url[0:4] == 'http:'and not self.isindexed(url):
                            newspages.add(url)
                        linkText = self.gettextonly(url)
                        self.addlinkref(page,url,linkText)
                        
                self.dbcommit()
            pages = newpages
  
    def createindextables(self):
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit()
        

mainFunction

In [29]:
import urllib2
from bs4 import BeautifulSoup,Tag,CData
from urlparse import urljoin

#构造一个单词列表，这些单词将被忽略
ignorewords = set(['the','of','to','and','a','in','is','it'])
pagelist = ['http://www.downloadsquad.com/rss.xml']
crawler = crawler('searchindex.db')
crawler.crawl(pagelist)


OperationalError: no such table: urllist

In [21]:
c = urllib2.urlopen('http://www.downloadsquad.com/rss.xml')
soup = BeautifulSoup(c.read(),"html5lib")
link = soup('a')
link[:10]

[<a class="" href="/">\n\t\t\t<h1>\n\t\t\t  <svg aria-label="Engadget" class="th-title header-logo" role="img">\n\t\t\t    <title>Engadget</title>\n\t\t\t    <use xlink:href="#icon-logo"></use>\n\t\t\t  </svg>\n\t\t\t</h1>\n\t\t\t</a>,
 <a class="increase-hit th-meta" data-behavior="Pop" data-engadget-popname="engadget_auth_window" href="https://www.engadget.com/user/login" target="_blank">            <span class="increase-hit__inner">\n                            Login\n                          </span>\n            </a>,
 <a class="hide@tl+ table-cell ta-c c-gray-5 w-60@s w-80@m+ full-height border-left" data-search-box-trigger="" href="#search-box">\n          <svg class="icon inline-block vm"><use xlink:href="#icon-search"></use></svg>\n      </a>,
 <a class="i-nav_drawer_slide@tp-__close hide@tl+ absolute r-n40 c-gray-2" href="#">\n    <svg class="icon increase-hit__inner">\n      <use xlink:href="#icon-close"></use>\n    </svg>\n  </a>,
 <a class="increase-hit@tl+ c-gray-10:hvr b

In [23]:
class searcher:
    def __init__(self,dbname):
        self.con = sqlite.connect(dbname)
        
    def __del__(self):
        self.con.close()
        
    def getmatchrows(self,q):
        #构造查询的字符串:
        fieldlist  = 'w0.urlid'
        tablelist  = ''
        clauselist = ''
        wordids = []
        
        #根据空格拆分单词
        words = q.split(' ')
        tablenumber = 0
        
        for word in words:
            #获取单词id
            wordrow = self.con.excute("select rowid from wordlist where word = '%s'" % word).fetchone()
            if wordrow != None:
                wordid = wordrow[0] #wordrow 是一个元祖
                wordids.append(wordid)
                if tablenumber >0:
                    tablelist += ','
                    clauselist += ' and '
                    clauselist += 'w%d.urlid = w%d.urlid and '%(tablenumber-1,tablenumber)
                fieldlist += ',w%d.location'% tablenumber
                tablelist += 'wordlocation'% tablenumber
                clauselist += 'w%d.wordid = %d' % (tablenumber,wordid)
                tablenumber += 1
            
            #根据各个组分,建立查询
            fullquery = 'select %s from %s where %s'%(fieldlist,tablelist,clauselist)
            cur = self.con.excute(fullquery)
            rows = [row for row in cur]
            
            return rows,wordids
        
                

In [ ]:
e = searcher('seacheindex.db')
e.getmatchchrows('')